## Используется гибридный подход: эвристика + токенизатор BERT. ##
## в Kaggle использовался GPU P100 ##

Программа использует гибридный подход для восстановления пробелов: сначала определяет обязательные границы между разными типами символов (цифры/буквы, кириллица/латиница), затем внутри полученных блоков применяет динамическое программирование для оптимального разбиения на слова. Стоимость слов оценивается через BERT-токенизатор (количество субтокенов) или эвристики, минимизируя общую стоимость разбиения. Алгоритм учитывает специфику объявлений через словарь частых слов и лингвистические паттерны.

In [1]:
!pip install wordfreq transformers torch 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import os
import re
import sys
import math
import json
import gc
import pandas as pd
import csv

Инициализация токенизатора, проверка на то, что он действительно запустился и работает. Rubert-tiny2 стал оптимальным выбором, так как он маленький и поддерживает русский язык

In [3]:
HF_TOKENIZER = None
try:
    from transformers import AutoTokenizer
    try:
        HF_TOKENIZER = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2", use_fast=True)
    except Exception as e:
        print(f"[warn] Can't load tokenizer cointegrated/rubert-tiny2: {e}")
        HF_TOKENIZER = None
except Exception as e:
    print(f"[warn] transformers not available: {e}")
    HF_TOKENIZER = None

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### Эвристический оценщик стоимости слов на основе лингвистических паттернов. ### 
Используется когда BERT-токенизатор недоступен. Очень простой вариант «токенизатора-оценщика стоимости». Он имитирует: чем «естественнее» строка (согласные/гласные чередуются, есть суффиксы/окончания и т.п.), тем меньше «стоимость». Это лишь эвристика на случай отсутствия HF-модели. ( в данном коде не используется, был выполнен в качестве практики, да, не использует CPU, но не самые лучшие результаты). Была идея выполнить задание с помощью парса википедии или использования тайги, но всё начало упираться в их размер, из-за чего фокус внимая был смещён на нейросетевые модели

In [4]:
#HF_TOKENIZER = None
#раскомментите, если хотите протестировать без BERT

In [5]:
class FallbackTokenizer:
    VOWELS = set(list("аеёиоуыэюяAEЁIOUYaeiouy"))
    # часто встречающиеся русских окончания, для проверки
    SUFFIXES = ("ами", "ями", "ами", "ием", "ыми", "ими", "ого", "ему", "ым", "им", "ой", "ей", "ий", "ый", "ой", "ая", "яя", "ые", "ие", "ам", "ям", "ах", "ях", "ов", "ев", "ом", "ем", "ам", "ям", "ть", "ти", "ть", "ться", "ться", "ние", "ция", "ская", "ский")

    def tokenize_cost(self, w: str) -> float:
        # Базовая стоимость: примерно 1 на 4-5 символов
        base = max(1.0, len(w) / 5.0)
        # Бонус за наличие гласных (избегаем большого количества согласных вместе (больше 5 встречается очень редко))
        if any(ch in self.VOWELS for ch in w.lower()):
            base -= 0.2
        penalty_cc = 0
        cons_run = 0
        for ch in w.lower():
            if ch.isalpha() and ch not in self.VOWELS:
                cons_run += 1
                if cons_run >= 3:
                    penalty_cc += 0.2
            else:
                cons_run = 0
        base += penalty_cc
        
        # Бонус за суффиксы
        wl = w.lower()
        if any(wl.endswith(suf) for suf in self.SUFFIXES):
            base -= 0.2
            
        # [0.5, 6] для стабильности
        return float(min(6.0, max(0.5, base)))

In [6]:
class TokenCost:
    def __init__(self, hf_tokenizer):
        self.hf = hf_tokenizer
        self.fallback = FallbackTokenizer()

        self.frequent_small = {
            "и","в","во","на","с","со","к","ко","о","об","обо","от","до","по","за","у","из","изo","для","при","над",
            "под","перед","через","про","без","между","около","после","как","что","где","да","но","или","либо",
            "ли","же","бы","не","ни","то","же","это","эта","этот","эти","там","тут","только","уже","ещё","ещё",
            "а","я","мы","вы","он","она","они","его","её","их","мой","моя","мои","твой","твоя","твои",
            # лексика, больше свойственная именно Авито
            "куплю","продам","продаю","отдам","даром","ищу","срочно","недорого","доставка","б/у","новый","новая","новое",
            "айфон","iphone","про","макс","диван","шкаф","монитор","дюймов","коврик","йоги","квартира","дом","комната",
            "мебелью","техникой","состоянии","хорошем","почти","с","зеркалами"
        }

        # Разрешенные односимвольные слова (без сильного штрафа)
        self.allowed_single = {"и","в","к","с","у","я","а","о"}

    def cost(self, w: str) -> float:
        if not w:
            return 0.0
        wl = w.lower()

        if self.hf is not None:
            try:
                toks = self.hf.tokenize(" " + w)
                base = float(len(toks))
                # нормализация
                base = min(6.0, max(0.5, base))
            except Exception:
                base = self.fallback.tokenize_cost(w)
        else:
            base = self.fallback.tokenize_cost(w)

        # Скидка для частых коротких слов
        if wl in self.frequent_small:
            base -= 0.4

        # Штраф за одиночную букву, если она не в списке разрешенных
        if len(w) == 1 and wl not in self.allowed_single and w.isalpha():
            base += 0.6

        # Чуть поощрим длины 2-7 символов, на первый взгляд +/- аткая длина слов в текстовых данных
        if 2 <= len(w) <= 7:
            base -= 0.1

        return float(min(6.0, max(0.3, base)))

token_cost = TokenCost(HF_TOKENIZER)

### Вспомогательные функции ###

In [7]:
CYR = re.compile(r"[А-Яа-яЁё]")
LAT = re.compile(r"[A-Za-z]")
DIG = re.compile(r"[0-9]")

def is_cyr(ch): return bool(CYR.fullmatch(ch))
def is_lat(ch): return bool(LAT.fullmatch(ch))
def is_dig(ch): return bool(DIG.fullmatch(ch))
def is_letter(ch): return ch.isalpha() 
def is_punct(ch):
    # Разделительная пунктуация, после которой обычно ставят пробел
    return ch in ",;:!?"

def need_space_after_punct(ch):
    return ch in ",;:!?"

def need_hard_boundary(c1, c2):
    # Жесткая граница: туда точно ставим пробел
    # буква <-> цифра
    if (is_letter(c1) and is_dig(c2)) or (is_dig(c1) and is_letter(c2)):
        return True
    # кириллица <-> латиница
    if (is_cyr(c1) and is_lat(c2)) or (is_lat(c1) and is_cyr(c2)):
        return True
    # нижний -> верхний регистр в пределах букв
    if c1.isalpha() and c2.isalpha() and (c1.islower() and c2.isupper()):
        return True
    return False

### Сегментация одного блока без "жёстких" границ (возвращает список позиций, где ставить пробел внутри блока s)

In [8]:
def segment_block(s: str, max_word_len: int = 24):
    n = len(s)
    if n <= 1:
        return []

    # DP
    INF = 1e9
    dp = [INF] * (n + 1)
    bp = [-1] * (n + 1)
    dp[0] = 0.0

    # доп защита - не начинаем слово с апострофов/дефисов/знаков
    def is_good_start(ch):
        return ch.isalnum()

    for i in range(1, n + 1):
        best = INF
        best_j = -1
        j_start = max(0, i - max_word_len)
        for j in range(j_start, i):
            w = s[j:i]
            if not is_good_start(w[0]):
                continue
            c = token_cost.cost(w)
            cand = dp[j] + c
            if cand < best:
                best = cand
                best_j = j
        dp[i] = best
        bp[i] = best_j

    # восстановление разбиений
    cuts = []
    i = n
    while i > 0 and bp[i] != -1:
        j = bp[i]
        if j > 0:
            cuts.append(j)
        i = j
    cuts.sort()
    return cuts

In [9]:
def predict_space_positions(text: str):
    n = len(text)
    if n == 0:
        return []

    # Сначала «жесткие» позиции пробелов:
    hard_spaces = set()

    # Пробел после разделительных знаков ,;:!? если далее буква/цифра
    for i, ch in enumerate(text):
        if need_space_after_punct(ch) and i + 1 < n and (text[i+1].isalnum() or text[i+1] in "([{"):
            hard_spaces.add(i + 1)

    # Переходы, где обязательно разделять
    for i in range(n - 1):
        c1, c2 = text[i], text[i + 1]
        if need_hard_boundary(c1, c2):
            hard_spaces.add(i + 1)

    block_starts = [0]
    for p in sorted(hard_spaces):
        if p not in block_starts:
            block_starts.append(p)
    block_starts = sorted(set(block_starts))
    if block_starts[-1] != n:
        block_starts.append(n)

    result_spaces = set(hard_spaces)

    for bi in range(len(block_starts) - 1):
        start = block_starts[bi]
        end = block_starts[bi + 1]
        if start >= end:
            continue
        block = text[start:end]

        # Если блок очень короткий — ничего не делим
        if len(block) <= 2:
            continue

        # Не запускаем на блоках, которые по сути "склеены" пунктуацией с двух сторон, но на алфавитно-цифровых блоках — запускаем
        if any(ch.isalnum() for ch in block):
            local_cuts = segment_block(block, max_word_len=24)
            for c in local_cuts:
                result_spaces.add(start + c)

    # Чистка пробелов в конце и в начале
    result = [p for p in sorted(result_spaces) if 0 < p < n]

    # + не ставим сразу два пробела подряд (типа после запятой + разбиения)
    cleaned = []
    last = -10
    for p in result:
        if p == last:
            continue
        cleaned.append(p)
        last = p

    return cleaned

### Обработка CSV с исправлением проблемы парсинга ###
Запятые в текстовых данных портили всю картину, было принято решение объединять всё после id

In [10]:
def safe_read_csv(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)
        
        for i, row in enumerate(reader):
            if len(row) == 2:
                # нормальный случай
                data.append(row)
            elif len(row) > 2:
                # случай с лишними запятыми в тексте
                id_val = row[0]
                text_val = ','.join(row[1:])
                data.append([id_val, text_val])
            else:
                # если строка пустая или с одним элементом, проверка на всякий случай
                print(f"Warning: строка {i+2} имеет неверный формат: {row}")

    df = pd.DataFrame(data, columns=header)
    return df

In [11]:
def run_inference_csv(input_path: str, output_path: str = "submission.csv",
                      text_col: str = "text_no_spaces", id_col: str = "id"):
    
    # Безопасное чтение CSV
    try:
        df = safe_read_csv(input_path)
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")

    text_col = df.columns[-1]
    
    # Обработка данных
    preds = []
    for idx, row in df.iterrows():
        raw = str(row[text_col]) if pd.notna(row[text_col]) else ""
        # уберем явные пробелы, если они есть в тексте (могут быть ошибочными)
        raw_nospace = raw.replace(" ", "")
        pos = predict_space_positions(raw_nospace)
        preds.append(json.dumps(pos, ensure_ascii=False))

    out_df = pd.DataFrame({
        'id': df[id_col] if id_col in df.columns else range(len(df)),
        'predicted_positions': preds
    })
    
    out_df.to_csv(output_path, index=False)
    print(f"[info] saved to {output_path}, строк: {len(out_df)}")


In [12]:
inp = "/kaggle/input/avito-ds-show/dataset_1937770_3.txt"
outp = "/kaggle/working/submission.csv"
print(f"Обработка файла: {inp}")
print(f"Выходной файл: {outp}")
run_inference_csv(inp, outp)

Обработка файла: /kaggle/input/avito-ds-show/dataset_1937770_3.txt
Выходной файл: /kaggle/working/submission.csv
[info] saved to /kaggle/working/submission.csv, строк: 1005


## Тест без BERT ##
не используется, но интересно посмотреть, что получилось))

In [13]:
tests = [
        "книгавхорошемсостоянии",
        "куплюайфон14про", 
        "ищудомвПодмосковье",
        "сдаюквартирусмебельюитехникой",
        "новыйдивандоставканедорого",
        "отдамдаромкошку",
        "работавМосквеудаленно",
        "99,продамшкаф,почтиновый,сзеркалами",
        "ищукнигубратьякарамазовы,срочно",
        "новыймонитор,27дюймов,доставка",
        "куплюковрикдляйоги,недорого!"
    ]

In [14]:
#Для отладки
def apply_spaces(text: str, positions):
    if not positions:
        return text
    res = []
    pos_set = set(positions)
    for i, ch in enumerate(text):
        res.append(ch)
        if (i + 1) in pos_set:
            res.append(" ")
    return "".join(res)

In [15]:
print("=== ТЕСТ БЕЗ BERT ===")
HF_TOKENIZER = None
token_cost = TokenCost(HF_TOKENIZER)
for text in tests:
    result = apply_spaces(text, predict_space_positions(text))
    print(f"{text} -> {result}")

=== ТЕСТ БЕЗ BERT ===
книгавхорошемсостоянии -> книгав хорошем состоянии
куплюайфон14про -> куплю айфон 14 про
ищудомвПодмосковье -> ищу домв Подмо сковье
сдаюквартирусмебельюитехникой -> сдаюк варти русме бельюи техникой
новыйдивандоставканедорого -> новый диван доставка недорого
отдамдаромкошку -> отдам даром кошку
работавМосквеудаленно -> работав Моск веуда ленно
99,продамшкаф,почтиновый,сзеркалами -> 99, продам шкаф, почти новый, сзерк алами
ищукнигубратьякарамазовы,срочно -> ищу книгу брать якарам азовы, срочно
новыймонитор,27дюймов,доставка -> новый монитор, 27 дюймов, доставка
куплюковрикдляйоги,недорого! -> куплю коврик для йоги, не до рого!
